In [2]:
import os 
import pandas as pd
import itertools
import scipy.stats as stats
from favapy import fava
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import numpy as np 
import fnmatch
import glob

In [2]:
from glob import glob

In [3]:
def Metrics(data, minimum=0):
    pd.set_option('mode.chained_assignment', None)
    metrics_dict = {}
    SCORES = ("SCORE.1_TYPE","SCORE.2_TYPE","SCORE.3_TYPE","SCORE.4_TYPE","SCORE.5_TYPE")
    for index_file in glob.glob(f"{data}/*index*"):
        Index = pd.read_csv(index_file, sep='\t')
        for col in Index.columns:
            if col in SCORES:
                for index, value in Index[col].items():
                    if value == '-':
                        pass
                    elif value in metrics_dict:
                        metrics_dict[value] += 1
                    else:
                        metrics_dict[value] = 1
    metrics = pd.DataFrame({'Scoring': list(metrics_dict.keys()), 'Screens': list(metrics_dict.values())})
    metrics = metrics[metrics['Screens'] >= minimum]
    return metrics

In [3]:


def read_organism_file(organism_file):
    """Read the organism file and return a dictionary that maps aliases to official symbols."""
    dict = {}
    with open(organism_file) as f:
        next(f)
        for line in f:
            a = line.split("\t")
            dict[a[1]] = a[0]
        return dict


def filter_genes(df, gene_dict):
    """Filter the rows of a dataframe that correspond to genes in the gene dictionary."""
    identifiers_1 = set(df["OFFICIAL_SYMBOL"])
    test = set(gene_dict.keys()) & identifiers_1        
    df = df[df["OFFICIAL_SYMBOL"].isin(test)]
    df = df.dropna(subset=["OFFICIAL_SYMBOL"])
    df = df[df["OFFICIAL_SYMBOL"].apply(lambda x: isinstance(x, str))]

    return df

def Table(Score_Dic):
    # Create an empty dictionary to hold each gene's scores for all keys
    scores_dict = {}
    
    # Loop over each key and populate the dictionary
    for key, df in Score_Dic.items():
        # Create a dictionary with gene names as keys and corresponding scores as values
        gene_scores = {gene: score for gene, score in zip(df['Gene'], df['Score'])}
        
        # Update the main dictionary with the new key-value pairs
        scores_dict[key] = gene_scores
    
    # Create a dataframe from the dictionary
    Matrix = pd.DataFrame.from_dict(scores_dict, orient='columns')
    Matrix.index.name = None
    
    Matrix = Matrix.replace('-', np.nan)
    return Matrix

def read_score_files(data, method='None', Norm="None", organism_file="STRING proteins//9606.protein.aliases.v11.5.txt"):
    """Read the score files and return a dictionary of dataframes with gene names and scores."""
    gene_dict = read_organism_file(organism_file)
    Score_Dic = {}
    Dic = {}
    
    
    for k in glob(os.path.join(data, "*.index.*")):
        Index = pd.read_csv(k, sep='\t')
        SCORES = pd.DataFrame({
            "SCREEN_ID": Index["#SCREEN_ID"],
            "SCORE.1": Index["SCORE.1_TYPE"],
            "SCORE.2": Index["SCORE.2_TYPE"],
            "SCORE.3": Index["SCORE.3_TYPE"],
            "SCORE.4": Index["SCORE.4_TYPE"],
            "SCORE.5": Index["SCORE.5_TYPE"]
        })
        for m in method:
            for col in SCORES.columns[1:]:
                rows = SCORES[SCORES[col] == m]
                for screen_id in rows["SCREEN_ID"]:
                    if screen_id not in Dic.keys():
                           Dic[screen_id] = col
    Min = 0
    Max = 0
    for i in Dic.keys():
        pattern = "*" + str(i) + "-1.1.13.screen.*"
                        
        
        for filename in os.listdir(data):
            if fnmatch.fnmatch(filename, pattern):
                df = pd.read_csv(data + "//" + filename, sep='\t')
                df = filter_genes(df, gene_dict) 
                    
                if df['#SCREEN_ID'].iloc[0] in Dic:
                    
                    Scores = pd.to_numeric(df[Dic[df['#SCREEN_ID'].iloc[0]]], errors='coerce')
                    
        #######################################################################################################
                # Normalization approaches if any were selected
                    
                    
                    
                    if Norm == "None":

                        df_info = pd.DataFrame()
                        df_info["Gene"] = df["OFFICIAL_SYMBOL"]
                        df_info["Score"] = Scores
                        
                        Score_Dic[df['#SCREEN_ID'].iloc[0]] = df_info
                        
                    elif Norm == "AbsZScore":
                
                         
                        df_info = pd.DataFrame()
                        df_info["Gene"] = df["OFFICIAL_SYMBOL"]
                        df_info["Score"] = abs(stats.zscore(Scores))
                        
                        Score_Dic[df['#SCREEN_ID'].iloc[0]] = df_info
                
                    elif Norm == "Scaled":
                        
                        Scaled = Scores.values  
                        Scaled -= Scaled.min()  # equivalent to df = df - df.min()
                        Scaled /= Scaled.max()  
                        df_Scaled = pd.DataFrame(Scaled)
                        df_info = pd.DataFrame()
                        df_info["Gene"] = df["OFFICIAL_SYMBOL"]
                        df_info["Score"] = df_Scaled[0]    
                        
                        Score_Dic[df['#SCREEN_ID'].iloc[0]] = df_info  
                        
                    elif Norm == "CenterScaled":
                        Scaled = stats.zscore(Scores.values)
                        if Scaled.min() <= Min:
                            Min = Scaled.min()
                        if Scaled.max() >= Max:
                            Max = Scaled.max()
                           
                        
    
    if Norm == "CenterScaled":
        for i in Dic.keys():
            pattern = "*" + str(i) + "-1.1.13.screen.*"
            for filename in os.listdir(data):
                if fnmatch.fnmatch(filename, pattern):
                    df = pd.read_csv(data + "//" + filename, sep='\t')
                    df = filter_genes(df, gene_dict) 
                   
                    #for i in df["OFFICIAL_SYMBOL"]:
                     #   if type(i) == float:
                      #      print(i)
                    if df['#SCREEN_ID'].iloc[0] in Dic:
                        Scores = pd.to_numeric(df[Dic[df['#SCREEN_ID'].iloc[0]]], errors='coerce')
                        Scaled = stats.zscore(Scores.values)
                        Scaled -= Min  # equivalent to df = df - df.min()
                        Scaled /= Max 
                            
                        df_Scaled = pd.DataFrame(Scaled)
                        
                        df_info = pd.DataFrame()
                        df_info["Gene"] = df["OFFICIAL_SYMBOL"]
                        df_info["Score"] = df_Scaled[0] 
                        Score_Dic[df['#SCREEN_ID'].iloc[0]] = df_info
    
    Matrix = Score_Dic                         
    Matrix = Table(Score_Dic) 
    Matrix = Matrix.dropna(axis=0,thresh=1)
    #if Norm == "CenterScaled":

     #   center = 0
      #  center -= Min
       # center /= Max
        #Matrix = Matrix.fillna(center)
    
    return Matrix
                

In [4]:
def HisPlot(data,xmin = 0,xmax = 0):
    N_Screens = len(data.columns)
    palette = itertools.cycle(sns.color_palette("pastel"))
    plt.figure(figsize=(15, 12))
    plt.subplots_adjust(hspace=0.5)
    plt.title('Normalized Scores')
    for i in data:  
        df = data[i].dropna()
        sns.histplot(df, bins = 20,element="bars",color=next(palette)) 
        if xmin != 0 and xmax != 0:
            plt.xlim(xmin, xmax)

    return N_Screens

Benchmarking annotation prep

In [3]:
def Annot_network(network = 'Homo_Sapiens_Scaled.txt',alias = "STRING proteins//9606.protein.aliases.v11.5.txt"):
    dict = {}
    with open(alias) as f:
        next(f)
        for line in f:
            a = line.split("\t")
            dict[a[1]] = a[0]
    network = pd.read_csv(network,sep="\t", header = 0)

    identifiers_1 = [dict.get(key) for key in network.iloc[:,1]]
    identifiers_1_df = pd.DataFrame(identifiers_1)

    identifiers_2 = [dict.get(key) for key in network.iloc[:,2]]
    identifiers_2_df = pd.DataFrame(identifiers_2)

    network_ensps = pd.concat([identifiers_1_df,identifiers_2_df],axis=1)
    final_pairs = pd.concat([network_ensps,network.iloc[:,3]],axis=1)
    final_network = final_pairs.dropna()
    
    return final_network

# Creating and Running Networks

Homo_Sapiens ; Sacharomyces_cerevisiae

In [4]:
import glob
Metrics("Biogrid Data//Homo_Sapiens",0)

,Scoring,Screens
0,Log10 (Corrected p-Value),4
1,Gamma (normalized log2e/t),2
2,Bayes Factor,371
3,CasTLE Score,31
4,NES (Normalized enrichment score),6
5,STARS Score,46
6,CRISPR Score (CS),83
7,MaGeCK Score,158
8,Mean Depletion,3
9,Essentiality Score,3


In [289]:
from glob import glob
FAVA_Matrix = read_score_files("Biogrid Data//Homo_Sapiens",["Log2FC"],"CenterScaled","STRING proteins//9606.protein.aliases.v11.5.txt")

## Achilles

In [4]:
FAVA_Matrix = pd.read_csv('Achilles_Matrices//AvanaRawCounts_CorrelationMatrix.txt', sep='\t', index_col = 0)


In [11]:
FAVA_Matrix = df

In [4]:
df = pd.read_csv('GeneDepAchilles.txt', sep='\t', index_col = 1)
df = df.drop('Unnamed: 0.1', axis=1)
df.index = df.index.values
FAVA_Matrix = df

# All against all Pearson

In [6]:
df = df.transpose()

In [7]:
df

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,TRIM64B,TSGA10IP,TVP23C,TYW1B,WDR49,ZNF254,ZNF365,ZNF525,ZNF626,ZNF705D
SC-000004.AV01,0.006999,0.021605,0.193413,0.041025,0.006603,0.073499,0.003005,0.353604,0.003634,0.007883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC-000005.AV01,0.026153,0.015360,0.030870,0.016074,0.019305,0.094428,0.007682,0.288223,0.024913,0.025690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC-000007.AV01,0.016894,0.035978,0.033399,0.014054,0.006001,0.007675,0.024647,0.378520,0.044412,0.003625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC-000009.AV01,0.008225,0.033110,0.032229,0.033191,0.002756,0.036849,0.021905,0.744062,0.006473,0.026742,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC-000011.AV01,0.006407,0.014740,0.025318,0.003735,0.010439,0.162741,0.014212,0.136779,0.007661,0.011803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SC-002296.KY01,0.083755,0.016073,0.013664,0.011685,0.055022,0.019105,0.015512,0.740881,0.022968,0.004415,...,0.043013,0.289384,0.024052,0.320803,0.004215,0.014620,0.007635,0.007181,0.005348,0.083566
SC-002297.KY01,0.025275,0.335812,0.005773,0.010614,0.240726,0.042173,0.041938,0.346151,0.016078,0.004174,...,0.044780,0.245918,0.122149,0.212021,0.010240,0.008688,0.034163,0.003222,0.010777,0.210458
SC-002298.KY01,0.129833,0.138097,0.004433,0.023475,0.057158,0.035104,0.059610,0.016994,0.019365,0.073161,...,0.138939,0.460446,0.473117,0.044444,0.004826,0.019404,0.026582,0.005012,0.003349,0.472797
SC-002304.KY01,0.133953,0.004651,0.017348,0.003651,0.053945,0.064234,0.009553,0.307059,0.125403,0.001917,...,0.019182,0.295925,0.533357,0.089729,0.006463,0.027486,0.028291,0.003580,0.014043,0.154240


In [ ]:
# Compute the correlation matrix using Pearson correlation coefficient
corr_matrix = df.corr(method='pearson')

# Set a threshold for the correlation coefficient
threshold = 0.95

# Filter the correlation matrix to only include pairs with correlation coefficient above the threshold
high_corr_pairs = corr_matrix[abs(corr_matrix) > threshold].stack().reset_index()

# Rename the columns of the filtered pairs dataframe
high_corr_pairs.columns = ['Protein_1', 'Protein_2', 'Score']


In [ ]:
high_corr_pairs

In [16]:
Network = high_corr_pairs 

In [ ]:
HisPlot(FAVA_Matrix,0,0)

In [29]:
def FAVA(data):

    my_FAVAourite_network = fava.cook(data = data,
                                  log2_normalization = False, # If your data are normalized set this to False
                                  hidden_layer = 1000, # If None, it will be adjusted base on the input size.
                                  latent_dim = 100, # If None, it will be adjusted base on the size of the hidden layer.
                                  epochs = 10, 
                                  batch_size = 32,
                                  PCC_cutoff = 0.5) # This is arbitraty. We need to plot the score distribution.
    return my_FAVAourite_network

In [ ]:
Network = FAVA(FAVA_Matrix)


Epoch 1/10
572/572 [==============================] - 34s 56ms/step - loss: 13.1206 - accuracy: 0.6503 - val_loss: 7.3061 - val_accuracy: 0.6554
Epoch 2/10
572/572 [==============================] - 33s 59ms/step - loss: 6.0505 - accuracy: 0.6604 - val_loss: 4.8612 - val_accuracy: 0.6584
Epoch 3/10
572/572 [==============================] - 34s 60ms/step - loss: 5.0666 - accuracy: 0.6628 - val_loss: 7.5831 - val_accuracy: 0.6649
Epoch 4/10
572/572 [==============================] - 31s 54ms/step - loss: 4.1839 - accuracy: 0.6647 - val_loss: 3.9035 - val_accuracy: 0.6657
Epoch 5/10
572/572 [==============================] - 31s 54ms/step - loss: 4.1966 - accuracy: 0.6665 - val_loss: 3.7219 - val_accuracy: 0.6685
Epoch 6/10
572/572 [==============================] - 31s 55ms/step - loss: 3.8679 - accuracy: 0.6681 - val_loss: 3.4348 - val_accuracy: 0.6705
Epoch 7/10
572/572 [==============================] - 32s 55ms/step - loss: 3.7504 - accuracy: 0.6694 - val_loss: 3.4044 - val_accuracy

In [ ]:
Network

In [ ]:
Network.to_csv('Homo_Sapiens.txt',sep='\t')
final_network = Annot_network('Homo_Sapiens.txt',"STRING proteins//9606.protein.aliases.v11.5.txt")
final_network.to_csv('Networks//Homo_Sapiens.tsv',sep='\t',header = False)
CommandLine()
from IPython.display import Image
Image("Roc_png//Homo_Sapiens_benchmark_kegg.png")

In [ ]:
Network.to_csv('Homo_Sapiens_Scaled.txt',sep='\t')

In [12]:
final_network = Annot_network('Homo_Sapiens_Scaled.txt',"STRING proteins//9606.protein.aliases.v11.5.txt")

In [ ]:
final_network.to_csv('Networks//Homo_Sapiens_Log2_Log2FC_Scaled.tsv',sep='\t',header = False)

Dont forget to change benchmark file when going from Human to Yeast

In [6]:
def CommandLine():
    !awk -F"\t" '{print $0, "FAVA"}' 'Networks//Homo_Sapiens.tsv' > tmp.tsv && mv tmp.tsv 'Dummies//Homo_Sapiens.tsv'
    !awk '{print $2,$3,$4,$5}' 'Dummies//Homo_Sapiens.tsv' > 'Dummies//Homo_Sapiens2.tsv'
    !python3 string_score_benchmark.py 'Dummies//Homo_Sapiens2.tsv' 'kegg_benchmarking.CONN_maps_in.v12.9606.tsv' 'none' > 'Benchmark_Kegg_Networks//Homo_Sapiens_benchmark_kegg.tsv'
    !Rscript benchmark_plot_coX.R 'Benchmark_Kegg_Networks//Homo_Sapiens_benchmark_kegg.tsv' 'Roc_png//Homo_Sapiens_benchmark_kegg.png'
    return

In [ ]:
CommandLine()

In [ ]:
from IPython.display import Image
Image("Roc_png//Homo_Sapiens_Log2_Log2FC_Scaled_benchmark_kegg.png")

In [1]:
%%capture --no-display

!pip install seaborn
!pip install favapy
!pip install scipy
!Rscript -e 'install.packages("plotly", repos="https://cloud.r-project.org")'
!Rscript -e 'install.packages("igraph", repos="https://cloud.r-project.org")'
